In [ ]:
import logging
from datetime import datetime

import fastf1 as ff1
import pandas as pd

from f1_analysis.fastf1_wrapper import FastF1Wrapper

logging.disable()

fastf1_wrapper = FastF1Wrapper()

season = 2025

events = []

for _, event in ff1.get_event_schedule(season, include_testing=False).iterrows():
    round_number = event["RoundNumber"]

    qualifying = ff1.get_session(season, round_number, "Q")

    if qualifying.date > datetime.now():
        continue

    practice_sessions = [ff1.get_session(season, round_number, "FP1")]

    if event["EventFormat"] == "conventional":
        practice_sessions.append(ff1.get_session(season, round_number, "FP2"))
        practice_sessions.append(ff1.get_session(season, round_number, "FP3"))

    events.append(
        {
            "event": event,
            "season": season,
            "qualifying": qualifying,
            "practice_sessions": practice_sessions,
        }
    )

In [ ]:
def load_fastest_quali_laps(qualifying):
    qualifying_laps = fastf1_wrapper.load_session_laps(qualifying)
    qualifying_laps["QualifyingTimeMs"] = (
        pd.to_timedelta(qualifying_laps["LapTime"]).dt.total_seconds() * 1000
    )
    qualifying_laps = qualifying_laps.dropna(subset=["QualifyingTimeMs"]).reset_index(
        drop=True
    )

    fastest_qualifying_laps_row_indices = qualifying_laps.groupby("Driver")[
        "QualifyingTimeMs"
    ].idxmin()
    fastest_qualifying_laps = qualifying_laps.loc[fastest_qualifying_laps_row_indices][
        ["Driver", "QualifyingTimeMs"]
    ]
    fastest_qualifying_laps = fastest_qualifying_laps.reset_index(drop=True)

    return fastest_qualifying_laps


def load_fastest_practice_laps(practices):
    practice_laps = pd.DataFrame()

    for practice in practices:
        practice_laps = pd.concat(
            [practice_laps, fastf1_wrapper.load_session_laps(practice)]
        )

    practice_laps["LapTimeMs"] = (
        pd.to_timedelta(practice_laps["LapTime"]).dt.total_seconds() * 1000
    )
    practice_laps["Sector1TimeMs"] = (
        pd.to_timedelta(practice_laps["Sector1Time"]).dt.total_seconds() * 1000
    )
    practice_laps["Sector2TimeMs"] = (
        pd.to_timedelta(practice_laps["Sector2Time"]).dt.total_seconds() * 1000
    )
    practice_laps["Sector3TimeMs"] = (
        pd.to_timedelta(practice_laps["Sector3Time"]).dt.total_seconds() * 1000
    )

    fastest_practice_laps = (
        practice_laps.groupby("Driver")
        .agg(
            {
                "Sector1TimeMs": "min",
                "Sector2TimeMs": "min",
                "Sector3TimeMs": "min",
                "LapTimeMs": "min",
            }
        )
        .reset_index()
    )

    fastest_practice_laps["SectorsSum"] = (
        fastest_practice_laps["Sector1TimeMs"]
        + fastest_practice_laps["Sector2TimeMs"]
        + fastest_practice_laps["Sector3TimeMs"]
    )

    return fastest_practice_laps


df = pd.DataFrame()

for event in events:
    round_number = event["event"]["RoundNumber"]

    fastest_qualifying_laps = load_fastest_quali_laps(event["qualifying"])
    fastest_qualifying_laps["RoundNumber"] = round_number

    fastest_practice_laps = load_fastest_practice_laps(event["practice_sessions"])
    fastest_practice_laps["RoundNumber"] = round_number

    event_data = pd.merge(
        fastest_qualifying_laps,
        fastest_practice_laps,
        on=["Driver", "RoundNumber"],
        how="left",
    )
    event_data["Season"] = event["season"]

    df = pd.concat([df, event_data])

df["SectorsQualiDelta"] = df["SectorsSum"] - df["QualifyingTimeMs"]
df["PracticeQualiDelta"] = df["LapTimeMs"] - df["QualifyingTimeMs"]

df[["LapTimeMs", "SectorsSum", "QualifyingTimeMs"]].corr()

In [ ]:
import plotly.graph_objects as go

df = df[
    (df["SectorsQualiDelta"].abs() < 2000) & (df["PracticeQualiDelta"].abs() < 2000)
]

sectors_quali_delta_trace = go.Box(
    name="Sectors vs. Qualifying",
    x=df["RoundNumber"],
    y=df["SectorsQualiDelta"],
    boxpoints="outliers",
    hovertext=df["Driver"],
)

layout = go.Layout(
    xaxis_title="Round",
    yaxis_title="Delta in MS",
    xaxis=dict(dtick=1),
    boxmode="group",
    height=800,
)

fig = go.Figure(data=[sectors_quali_delta_trace], layout=layout)
fig.show()

In [ ]:
sectors_quali_delta_trace = go.Box(
    name="Sectors vs. Qualifying",
    x=df["Driver"],
    y=df["SectorsQualiDelta"],
    boxpoints="outliers",
    hovertext=df["RoundNumber"],
)

fig = go.Figure(data=[sectors_quali_delta_trace], layout=layout)
fig.show()

In [ ]:
best_per_round = df.groupby("RoundNumber")["QualifyingTimeMs"].transform("min")
df["DeltaToQuickest"] = df["QualifyingTimeMs"] - best_per_round

mean_deltas = (
    df.groupby("Driver")
    .agg(
        {
            "PracticeQualiDelta": "mean",
            "SectorsQualiDelta": "mean",
            "DeltaToQuickest": "mean",
        }
    )
    .reset_index()
)

mean_deltas = mean_deltas.sort_values("SectorsQualiDelta", ascending=False)

mean_delta_trace = go.Bar(
    name="Sectors vs. Qualifying",
    x=mean_deltas["Driver"],
    y=mean_deltas["SectorsQualiDelta"],
)

mean_delta_to_pole_trace = go.Scatter(
    name="Delta to pole lap",
    x=mean_deltas["Driver"],
    y=mean_deltas["DeltaToQuickest"],
    mode="lines+markers",
    line_color="red",
)

fig = go.Figure(data=[mean_delta_trace, mean_delta_to_pole_trace], layout=layout)
fig.show()